<a href="https://colab.research.google.com/github/martinbremm/embedding-comparison/blob/main/BERT_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [59]:
!pip install transformers

In [60]:
import torch
from transformers import BertTokenizer, BertModel

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
cd /content/drive/MyDrive/Datasets

/content/drive/MyDrive/Datasets


# Importing and preprocessing data

In [63]:
import pandas as pd

df = pd.read_csv("data_frame_processed_labeled.csv")
# filtering out missing values in descriptions column
df = df[["clean_description", "clean_fn"]].dropna()

In [64]:
# examples of semantically- and acoustically-similar sound-producing mechanisms

# acoustically similar sound-producing mechanism
acoustic_dict = {
    "impact": ["bump", "drop", "fall", "bounce", "clap", "slam", "clang", "step", "punch"],
    "explosion": ["blast", "explode", "bang", "blowout", "burst"],
    "vocal": ["talk", "scream", "hiss", "yell", "chant", "conversation", "murmur", "shout"],
    "nose": ["wheeze", "sneeze", "snort", "snore", "sniff"],
}

# semantically similar sound-producing mechanism
semantic_dict = {
    "locomotion": ["walk", "gallop", "run" , "march", "jump", "move"],
    "dog": ["howl", "bark", "growl", "whimper", "scratch", "pant"],
    "vehicle": ["rev", "drive", "pass", "accelerate", "brake"],
    "complex_sound" : ["fight", "box", "destruction", "accident", "wrestle", "bowl", "exercise"]
}

In [65]:
def create_tuples(dictionary):
    dictionary_words = [word for words in dictionary.values() for word in words]  # list of words used as index
        
    category = []
    for key, val in dictionary.items():
        category.extend([key]*len(val)) # list of category memberships

    similarity = ["acoustic_sim" if dictionary=="acoustic_sim" else "semantic_sim" for word in dictionary_words]
        
    tuples = list(zip(similarity, category, dictionary_words))
    return tuples

In [66]:
acoustic_df = pd.DataFrame(data=create_tuples(acoustic_dict), columns = ["Similarity", "Category", "Word"])
semantic_df = pd.DataFrame(data=create_tuples(semantic_dict), columns = ["Similarity", "Category", "Word"])


combined_df = pd.concat([acoustic_df, semantic_df], ignore_index = True)

In [67]:
combined_df

,Similarity,Category,Word
0,semantic_sim,impact,bump
1,semantic_sim,impact,drop
2,semantic_sim,impact,fall
3,semantic_sim,impact,bounce
4,semantic_sim,impact,clap
5,semantic_sim,impact,slam
6,semantic_sim,impact,clang
7,semantic_sim,impact,step
8,semantic_sim,impact,punch
9,semantic_sim,explosion,blast


In [68]:
# tokenizing the sentences into single words
from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
tokens = df["clean_description"].apply(word_tokenize)
content = []
for word in combined_df["Word"]:
  word_df = df[tokens.apply(lambda sentence: word in sentence)]
  word_df.insert(0, "Word", [word]*len(word_df))
  word_df = word_df.reset_index(drop=True)
  content.append(word_df)
examples1 = pd.concat(content, ignore_index = True)

In [70]:
examples1

,Word,clean_description,clean_fn
0,bump,underwater bump bridge mono drops falls hits i...,hit_concrete
1,bump,pinball machine hitting knock bump amusement p...,pinball_machine
2,bump,pinball machine hitting knock bump hard amusem...,pinball_machine
3,bump,auto crash light bump hard object car crashes,car_crash
4,bump,auto crash light bump hard object car crashes,car_crash
...,...,...,...
74523,exercise,metal drop weight plate exercise sport impacte...,metal_drop
74524,exercise,metal drop weight plate exercise sport impacte...,metal_drop
74525,exercise,metal drop weight plate exercise sport impacte...,metal_drop
74526,exercise,metal drop weight plate exercise sport impacte...,metal_drop


In [71]:
content = []
for word in combined_df["Word"]:
  word_df = df[df["clean_description"].str.contains(" " + word + " ")]
  word_df.insert(0, "Word", [word]*len(word_df))
  word_df = word_df.reset_index(drop=True)
  content.append(word_df)
examples2 = pd.concat(content, ignore_index = True)

In [72]:
content = []
for word in combined_df["Word"]:
  word_df = df[df["clean_fn"].str.contains(word)]
  word_df.insert(0, "Word", [word]*len(word_df))
  word_df = word_df.reset_index(drop=True)
  content.append(word_df)
print(f"Contains examples of {len(content)} words!")

Contains examples of 51 words!


In [73]:
examples = pd.concat(content, ignore_index = True)
#examples.to_csv("BERT_example_sentences.csv")

In [74]:
examples

,Word,clean_description,clean_fn
0,bump,wood en plank skid drag along bumpy surface,skid_wood_wooden_plank_drag_along_bumpy
1,bump,wood en plank skid drag along bumpy surface,skid_wood_wooden_plank_drag_along_bumpy
2,bump,video game electronic bump,video_game_electronic_bump
3,bump,luggage rolling bumps stop,luggage_rolling_bump
4,bump,body bumps wall various,body_bumps_wall_var
...,...,...,...
57617,exercise,exercise equipment weight lifting competition ...,exercise_equipment
57618,exercise,exercise equipment weight lifting competition ...,exercise_equipment
57619,exercise,exercise weight room training lifting bars wei...,exercise_weight_room
57620,exercise,exercise weight room training lifting bars wei...,exercise_weight_room


Creating Dataframe with example sentences obtained from out dataset (randomly chosen sentences including the words defined in the dictionaries)

In [84]:
descriptions = df["clean_description"]
content = [descriptions[descriptions.str.contains(" " + word + " ")].sample(n=1, ignore_index=True)[0] for word in combined_df["Word"]]

combined_df["example_sentence"] = content

In [86]:
combined_df.head(n=5)

,Similarity,Category,Word,example_sentence
0,semantic_sim,impact,bump,metal drum bump dull
1,semantic_sim,impact,drop,plastic cap drop bounce medium sized
2,semantic_sim,impact,fall,metal fall stairs
3,semantic_sim,impact,bounce,car vw beetle turbo driver door various open c...
4,semantic_sim,impact,clap,several speed skaters wearing clap skates skat...


# BERT model preprocessing

In [87]:
# Adding BERT tokens
marked_text = combined_df["example_sentence"].apply(lambda sen: "[CLS] " + sen + " [SEP]")
marked_text

0                      [CLS] metal drum bump dull [SEP]
1      [CLS] plastic cap drop bounce medium sized [SEP]
2                         [CLS] metal fall stairs [SEP]
3     [CLS] car vw beetle turbo driver door various ...
4     [CLS] several speed skaters wearing clap skate...
5     [CLS] telephone domestic old dial phone slam r...
6              [CLS] impact metal ic clang reverb [SEP]
7     [CLS] footstep dress shoe single step shoes ha...
8     [CLS] vehicle motorcycle motorcycles honda val...
9     [CLS] aviation jet various take series engine ...
10    [CLS] science fiction various crunch take rock...
11    [CLS] door doors screen open close stretching ...
12    [CLS] sizzle match strike ignite flame blowout...
13          [CLS] low slow ed air bag burst heavy [SEP]
14    [CLS] sport soccer game medium distant predomi...
15    [CLS] animal scream hollow resonant like lowes...
16    [CLS] science fiction machines motors pass tak...
17    [CLS] siren emergency sirens police wail y

In [88]:
## Splitting sentence into tokens

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenized_text = marked_text.apply(lambda sen: tokenizer.tokenize(sen))
tokenized_text

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

0               [[CLS], metal, drum, bump, dull, [SEP]]
1     [[CLS], plastic, cap, drop, bounce, medium, si...
2                   [[CLS], metal, fall, stairs, [SEP]]
3     [[CLS], car, v, ##w, beetle, turbo, driver, do...
4     [[CLS], several, speed, skaters, wearing, clap...
5     [[CLS], telephone, domestic, old, dial, phone,...
6     [[CLS], impact, metal, ic, clan, ##g, rev, ##e...
7     [[CLS], foot, ##ste, ##p, dress, shoe, single,...
8     [[CLS], vehicle, motorcycle, motorcycles, hond...
9     [[CLS], aviation, jet, various, take, series, ...
10    [[CLS], science, fiction, various, crunch, tak...
11    [[CLS], door, doors, screen, open, close, stre...
12    [[CLS], si, ##zzle, match, strike, ign, ##ite,...
13    [[CLS], low, slow, ed, air, bag, burst, heavy,...
14    [[CLS], sport, soccer, game, medium, distant, ...
15    [[CLS], animal, scream, hollow, res, ##ona, ##...
16    [[CLS], science, fiction, machines, motors, pa...
17    [[CLS], siren, emergency, sirens, police, 

In [89]:
# Mapping token to BERT vocabulary index
indexed_tokens = tokenized_text.apply(lambda sen: tokenizer.convert_tokens_to_ids(sen))

In [90]:
# Zip words together with their indexes
token_id_tup = []
for ind, sen in enumerate(tokenized_text):
  token_id_tup.append(list(zip(sen, indexed_tokens[ind])))

In [91]:
# Zip words together with their indexes
for tup in token_id_tup[0]:
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))


[CLS]           101
metal         3,384
drum          6,943
bump         16,906
dull         10,634
[SEP]           102


In [92]:
# Creating segment IDs for BERT to distinguish between sentences
segment_ids = []
for ind, sen in enumerate(tokenized_text):
  segment_ids.append([ind] * len(sen))

In [93]:
tokens_tensor = [torch.tensor([token]) for token in indexed_tokens]
tokens_tensor[0]

tensor([[  101,  3384,  6943, 16906, 10634,   102]])

In [94]:
segments_tensor = [torch.tensor([id]) for id in segment_ids]
segments_tensor[0]

tensor([[0, 0, 0, 0, 0, 0]])

In [95]:
zipped_text_token_segment = list(zip(tokenized_text, tokens_tensor, segments_tensor))

for tup in zipped_text_token_segment:
  print(f"Sentence: {tup[0]}")
  print(f"Token_tensor: {tup[1]}")
  print(f"Segment_id: {tup[2]}")
  

Sentence: ['[CLS]', 'metal', 'drum', 'bump', 'dull', '[SEP]']
Token_tensor: tensor([[  101,  3384,  6943, 16906, 10634,   102]])
Segment_id: tensor([[0, 0, 0, 0, 0, 0]])
Sentence: ['[CLS]', 'plastic', 'cap', 'drop', 'bounce', 'medium', 'sized', '[SEP]']
Token_tensor: tensor([[  101,  6081,  6178,  4530, 17523,  5396,  7451,   102]])
Segment_id: tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
Sentence: ['[CLS]', 'metal', 'fall', 'stairs', '[SEP]']
Token_tensor: tensor([[ 101, 3384, 2991, 5108,  102]])
Segment_id: tensor([[2, 2, 2, 2, 2]])
Sentence: ['[CLS]', 'car', 'v', '##w', 'beetle', 'turbo', 'driver', 'door', 'various', 'open', 'close', 'occasional', 'bounce', 'back', 'liter', 'hp', 'cy', '##l', 'adaptive', 'trans', '[SEP]']
Token_tensor: tensor([[  101,  2482,  1058,  2860,  7813, 15386,  4062,  2341,  2536,  2330,
          2485,  8138, 17523,  2067, 23675,  6522, 22330,  2140, 19293,  9099,
           102]])
Segment_id: tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])

# Evaluating the BERT model

In [96]:
# suppressing the output with the magic command "capture"
%%capture

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

# setting model in eval mode (i.e., feed-forward operation)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [97]:
# apply BERT to text and collect all hidden states form the 12 BERT layers

with torch.no_grad():

  # evaluating the model only on the first pair of token tensor and segments tensor
  outputs = model(tokens_tensor[0], segments_tensor[0])


  # in our pretrained model, third item contains hidden states form all layers
  hidden_states = outputs[2]

In [98]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")

layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))

batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))

token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 6
Number of hidden units: 768


We want to group the values by tokens (i.e., parts of the words we want the embeddings for) and not by layer. Thus we stack all the layers together and extract the information across layers per token

In [99]:
# Concatenate the tensors for all layers. We use `stack` here to create a new dimension in the tensor
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 6, 768])

Current dimensions: [# layers, # batches, # tokens, # features] <br>
Desired dimensions: [# tokens, # layers, # features]

In [100]:
# Changing the order of dimensions by permutation
# Numbers in the permute function refer to the spot in the original list, i.e., the original index in the dimension ordering in token_embedding 
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([6, 13, 768])

First, let’s concatenate the last four layers, giving us a single word vector per token. Each vector will have length 4 x 768 = 3,072.

In [101]:
# stores token vectors
token_vecs_cat = []

for token in token_embeddings:

  # concatinating vectors from the last 4 model layers
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))


Shape is: 6 x 3072


# Putting it all together 
Creating word embedding for entire DataFrame

In [102]:
# apply BERT to text and collect all hidden states form the 12 BERT layers
token_vecs_cat = []


with torch.no_grad():
  # each tuple represents all the data for one of the 10 sentences
  for tup in list(zip(tokenized_text, tokens_tensor, segments_tensor)):

    # model gets tokens_tensor and segments_tensor as input, to create word embedding
    out = model(tup[1], tup[2])

  # in our pretrained model, third item contains hidden states form all layers
    hidden_states = out[2]


    # Concatenate the tensors for all layers. We use `stack` here to create a new dimension in the tensor
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    # stores token vectors
    for token in token_embeddings:

      # concatinating vectors from the last 4 model layers 
      # for dim=0 means only the token embeddings?
      cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

      # adding vector as a numpy array for visualization
      token_vecs_cat.append(cat_vec.numpy())

In [103]:
print(f"Number of all word embeddings: {len(token_vecs_cat)}")

Number of all word embeddings: 837


In [104]:
token_embeddings.size()

torch.Size([12, 13, 768])

In [105]:
token.size()

torch.Size([13, 768])

In [106]:
# getting all the tokens of all sentences into one list
tokens = []
for sentence in tokenized_text:
  tokens.extend(sentence)
print(f"Number of all tokens: {len(tokens)}")

Number of all tokens: 837


Showing the format of the output: The model gives us a single vector for each token, for which we printed only the 5 first dimensions

In [107]:
token_vecs_cat[0][:5]

array([-0.584233  ,  0.1512002 , -0.32292116, -0.04860656, -0.37233365],
      dtype=float32)

In [108]:
# creating df form dictionary with the dictionary keys (tokens) being the rows and the values (vector dimensions) being columns
data=dict(zip(tokens, token_vecs_cat))

vocab = pd.DataFrame.from_dict(data, orient="index")
vocab

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
[CLS],-0.385412,-0.169683,-0.122460,-0.285790,-0.188592,-0.052672,0.460083,0.186863,-0.518355,-0.160871,...,0.029798,-0.754809,-0.671109,-0.298449,0.553228,-0.415125,0.126625,0.135120,0.064915,0.228330
metal,0.526864,0.448068,0.339596,-0.115749,-0.187839,0.422706,0.492667,-0.261456,-0.046720,-0.407582,...,0.094306,-0.477550,1.421478,-0.274952,-0.303977,-0.500042,-1.314984,-0.962641,-0.074827,-0.493122
drum,0.480667,-0.239346,-0.138328,-0.254441,0.341284,0.460979,0.003544,0.598176,-0.872558,0.216899,...,0.641722,-0.622773,-0.153900,0.267731,0.585714,0.516981,-1.826138,0.697307,-0.144111,-1.094720
bump,-0.766358,0.150438,0.796784,-0.063850,0.813698,0.093535,-0.717201,0.465621,-0.285165,-0.140569,...,0.151924,-0.056756,1.593096,-0.314698,0.866845,0.262177,-1.890318,-0.288487,-0.182730,-0.318980
dull,-0.139497,-0.309761,-0.298775,0.184986,0.245184,0.126921,-0.054579,0.484003,-0.360832,-0.068242,...,0.206621,0.075714,0.165896,-0.041510,0.903641,-0.589952,0.644796,-0.068902,-0.902343,-0.279244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
electronic,0.043955,0.181946,0.234153,0.015311,1.420227,0.616279,0.644040,0.287112,-1.041873,-0.366492,...,0.441504,-0.521269,0.118978,0.249031,0.808547,-0.055141,-1.160134,-0.173069,0.224569,-0.349597
adjustment,-0.058595,-0.005818,0.014575,-0.067759,0.739163,0.284493,0.261438,0.282190,-1.255742,-0.069211,...,0.788132,-0.601480,0.218532,-0.531921,0.745811,0.050028,-0.653203,-0.762557,0.206812,-0.221271
start,-0.240616,0.112974,0.120422,-0.465654,1.167789,0.541881,0.388298,0.345745,-0.186948,0.051988,...,0.771203,-0.964499,-0.444635,0.125779,0.491638,1.040698,-0.225602,-0.737498,0.254134,-1.238674
climb,0.024471,0.046539,-0.066035,-0.178076,0.842693,0.365531,0.715804,-0.433530,-0.211332,-0.004604,...,0.385436,-0.326130,0.347607,-0.328896,0.484009,1.199531,-1.323713,-0.391217,-0.429910,0.235619


In [109]:
selected_vocab = vocab[vocab.index.isin(combined_df["Word"])]
selected_vocab

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
bump,-0.766358,0.150438,0.796784,-0.063850,0.813698,0.093535,-0.717201,0.465621,-0.285165,-0.140569,...,0.151924,-0.056756,1.593096,-0.314698,0.866845,0.262177,-1.890318,-0.288487,-0.182730,-0.318980
drop,0.378486,0.019752,0.233416,-0.287537,0.285330,-0.400916,-0.110548,0.206543,-0.343183,0.263094,...,0.430886,-0.840110,0.212373,-0.257885,0.251140,0.898735,-0.803699,-0.771321,-0.214084,-0.699663
bounce,0.267411,-0.189824,0.530648,-0.342058,0.242042,-0.760891,0.588256,0.157697,-0.273386,0.470276,...,-0.433407,0.305226,0.038115,0.492026,0.117041,0.213291,-1.355862,-0.976634,0.258117,-1.248368
fall,0.273359,0.546011,-0.097099,-0.239508,0.531610,0.064574,0.458318,0.270060,-0.100140,-0.037430,...,0.591706,-0.411014,0.885333,-0.731136,0.224104,0.372102,-0.872668,-1.180806,-0.397377,-0.526515
clap,0.724024,-0.546845,0.809024,-0.145585,0.463678,0.059112,-0.124074,0.556058,-0.791176,0.191562,...,0.528075,-0.391435,-0.638090,0.159265,0.887274,0.198440,-0.588360,0.144224,-0.079607,-0.978860
slam,0.862601,0.140036,0.585218,0.057647,0.243517,0.407262,0.092975,0.216761,-0.291751,-0.135581,...,1.281948,-0.580921,0.076436,0.297564,-0.302769,0.735943,0.020178,0.145076,0.509913,-1.625615
rev,0.354782,0.010428,0.660586,0.233403,0.211054,-0.327274,0.103515,0.096231,-0.059674,0.249406,...,0.738442,-0.392206,-1.600010,0.175493,0.517157,0.588347,-0.446151,-1.721137,-0.132160,-0.945002
step,0.231193,-1.113249,0.903277,0.117760,0.438051,-0.028522,-0.197499,-0.071361,0.192139,-0.470036,...,1.154288,-0.407733,-0.218205,-0.158187,0.286432,0.134104,-1.272135,-1.155807,0.275304,-1.451610
punch,-0.357430,0.348607,0.196897,-0.020377,-0.324706,-0.107266,0.274669,0.059066,0.236337,-0.556540,...,-0.136593,-0.089502,-0.951255,-0.571707,0.491681,0.790147,0.318136,-0.598648,0.766209,-0.546699
blast,0.275185,0.692628,0.199813,-0.300574,-0.141490,-0.268455,0.152974,0.471435,-0.144050,0.188913,...,0.274444,-0.607903,-0.789978,-0.318114,0.085406,-0.251474,-0.730113,-1.106835,-1.080569,-0.477772


In [110]:
tuples = [tuple(combined_df[combined_df["Word"]==i][["Category", "Word"]].values[0]) for i in selected_vocab.index]
multi_index = pd.MultiIndex.from_tuples(tuples, names=["Category", "Word"])

In [111]:
selected_vocab = selected_vocab.set_index(multi_index)
selected_vocab.sort_index(inplace=True)

In [112]:
selected_vocab

0         1         2         3         4     \
Category      Word                                                             
complex_sound accident     -0.071864 -0.430980  0.565145 -0.394851  0.830021   
              bowl          0.007823  0.084034 -0.283718 -0.260682 -0.047279   
              box           0.309398  0.243736  0.371980 -0.478570 -0.042897   
              destruction   0.425721  0.617503 -0.185585  0.166921 -0.005523   
              exercise      0.518139 -0.042294 -0.141330  0.037188  0.567031   
              fight        -0.181834  0.196767 -0.048378  0.199709 -0.129111   
              wrestle      -0.451489  1.022307 -0.052919 -0.040613 -0.182234   
dog           bark          0.009542  0.601970 -0.229793 -0.277801 -0.281056   
              growl         0.665008  0.316460  0.985501 -0.824503  0.330797   
              howl          0.326336  0.190100 -0.145694 -0.267369  0.343547   
              scratch       0.251591 -0.202100  0.147488 -0.349718  0.883501   
              whimper       0.101017 -0.011608  0.074293  0.067225  0.432380   
explosion     bang          0.090340  0.564852  0.363779 -0.602005  0.217588   
              blast         0.275185  0.692628  0.199813 -0.300574 -0.141490   
              burst         0.001529  0.378036  0.183695  0.029863  0.388109   
              explode       0.562243  1.010483  0.237769 -0.087924  0.998382   
impact        bounce        0.267411 -0.189824  0.530648 -0.342058  0.242042   
              bump         -0.766358  0.150438  0.796784 -0.063850  0.813698   
              clap          0.724024 -0.546845  0.809024 -0.145585  0.463678   
              drop          0.378486  0.019752  0.233416 -0.287537  0.285330   
              fall          0.273359  0.546011 -0.097099 -0.239508  0.531610   
              punch        -0.357430  0.348607  0.196897 -0.020377 -0.324706   
              slam          0.862601  0.140036  0.585218  0.057647  0.243517   
              step          0.231193 -1.113249  0.903277  0.117760  0.438051   
locomotion    jump          0.689748  0.444030  0.227783  0.031473  0.969267   
              march         0.333276  0.381269  0.276292 -0.071005  0.253495   
              move          0.169671 -0.311854  0.244828 -0.152413  0.381666   
              run           0.735289 -0.061865  0.135717  0.346581  0.410112   
              walk          0.508300 -0.205620  0.383891 -0.204555  0.502608   
nose          sniff         0.822967  0.283965  0.013353 -0.660593  0.135805   
              snort         0.234306  1.315447  0.176839 -0.217698 -0.049720   
vehicle       accelerate   -0.520190  0.460159  0.481030 -0.118447  0.190831   
              brake         0.065295  0.144953  0.899237 -0.528845  0.498808   
              drive         0.941188 -0.656077  1.128124  0.226752  0.571283   
              pass          0.412046 -0.165999  0.611774 -0.280603  0.390889   
              rev           0.354782  0.010428  0.660586  0.233403  0.211054   
vocal         chant         0.405199  0.096013 -0.048456 -0.301590 -0.765977   
              conversation -0.203793 -0.303932  0.487734  0.123119  0.030460   
              hiss          0.118874 -0.045100  0.116297 -0.165767 -0.141031   
              murmur        0.152099  0.313796  0.860306 -0.282433 -0.107641   
              scream        0.585470  0.054097  0.219685 -0.403706  0.198574   
              shout         0.426030  0.648306  0.558794 -0.479205  0.292200   
              talk          0.040241  0.163411  0.200503 -0.205080  0.309554   
              yell          0.281765  0.657102  0.880239 -0.986655  0.367361   

                                5         6         7         8         9     \
Category      Word                                                             
complex_sound accident      0.248474 -0.261721  1.118823 -0.413547  0.568947   
              bowl          0.364171 -0.124721  0.093506 -0.155283  0.133953   
              box          -0.0

In [117]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import zscore

# cosine similarity matrix
# 1-distance matrix with the cosine distance as a metric = similarity matrix
def create_similarity_matrix_df(df, metric):
    
    matrix = 1-squareform(pdist(zscore(df, axis=0), metric))

    matrix_df = pd.DataFrame(data=matrix, columns=multi_index, index=multi_index)
    matrix_df.sort_index(axis=0, inplace=True)
    matrix_df.sort_index(axis=1, inplace=True)

    # matrix_df.style.set_properties(**{'text-align': 'left'})
    return matrix_df.style.background_gradient(axis=None, cmap='RdBu') # changing color

In [118]:
create_similarity_matrix_df(selected_vocab, metric="cosine")